In [1]:
# Import packages
import pandas as pd
import numpy as np
from numpy import random as rn
rn.seed(1)

#### Input data

In [9]:
# Determine parameter
prod_cap = 150
c_setup = 100
c_inventory = 1
c_backlog = 3
L0 = 0
B0 = 0

#### Generate some samples and calculate the average total costs of all samples

In [10]:
n_repl = 10000
realized_cost = []
for _ in range(n_repl):
    demand = [20, 50, 10, 50, 50, 10, 20, 40, 20, 30]
    stoch_demand = [[10, 20, 30], [70, 50, 30], [25, 0, 5], [50, 50, 50], [70, 40, 40], [11, 9, 10], [23, 27, 10], [35, 42, 43], [10, 10, 40], [30, 30, 30]]
    realized_demand = [rn.choice(i) for i in stoch_demand]
    clsp = np.zeros(shape=(len(demand), 7))
    clsp[:, 0] = realized_demand
    clsp[:, 1] = [min(prod_cap, i) for i in demand]
    clsp[0, 2] = max(clsp[0, 1] - clsp[0, 0] + L0 - B0, 0)
    clsp[0, 3] = -min(clsp[0, 1] - clsp[0, 0] + L0 - B0, 0)
    for i in range(1, len(demand)):
        delta = clsp[i, 1] - clsp[i, 0] + clsp[i-1, 2] - clsp[i-1, 3]
        clsp[i, 2] = max(delta, 0)
        clsp[i, 3] = -min(delta, 0)
    clsp[:, 4] = [c_setup if i > 0 else 0 for i in clsp[:, 1]]
    clsp[:, 5] = clsp[:, 2] * c_inventory
    clsp[:, 6] = clsp[:, 3] * c_backlog
    total_cost = clsp[:, -3:].sum()
    
    realized_cost.append(total_cost)

print(f'Average expected cost: {np.mean(realized_cost)}')
print(f'CV: {np.std(realized_cost) / np.mean(realized_cost)}')

Average expected cost: 1381.7456
CV: 0.22439342190207728
